Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

display(namesDf.limit(10))

imdb_name_id name birth_name height bio birth_details date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children nm0000001 Fred Astaire Frederic Austerlitz Jr. 177 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. May 10, 1899 in Omaha, Nebraska, USA 1899-05-10 Omaha, Nebraska, USA June 22, 1987 in Los Angeles, California, USA  (pneumonia) 22.06.1987 Los Angeles, California, USA   pneumonia Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children) 2 0 1 2 nm0000002 Lauren Bacall Betty Joan Perske 174 Lauren Bacall was born Betty Joan Perske on September 16, 1924, in New York City. She was the daughter of Natalie Weinstein-Bacal, a Romanian Jewish immigrant, and William Perske, who was born in New Jersey, to Polish Jewish parents. Her family was middle-class, with her father working as a salesman and her mother as a secretary. They divorced when she was five and she rarely saw her father after that.As a school girl, she originally wanted to be a dancer, but later switched gears to head into acting. She studied at the American Academy of Dramatic Arts in New York, after attending She was educated at Highland Manor, a private boarding school in Tarrytown, New York (through the generosity of wealthy uncles), and then at Julia Richman High School, which enabled her to get her feet wet in some off-Broadway productions.Out of school, she entered modeling and, because of her beauty, appeared on the cover of Harper's Bazaar, one of the most popular magazines in the US. The wife of famed director Howard Hawks spotted the picture in the publication and arranged with her husband to have Lauren take a screen test. As a result, which was entirely positive, she was given the part of Marie Browning in Acque del Sud (1944), a thriller opposite Humphrey Bogart, when she was just 19 years old. This not only set the tone for a fabulous career but also one of Hollywood's greatest love stories (she married Bogart in 1945). It was also the first of several Bogie-Bacall films.After 1945's L'agente confidenziale (1945), Lauren received second billing in Il grande sonno (1946) with Bogart. The mystery, in the role of Vivian Sternwood Rutledge, was a resounding success. Although she was making one film a year, each production would be eagerly awaited by the public. In 1947, again with her husband, Lauren starred in the thriller La fuga (1947). The film kept movie patrons on the edge of their seats. The following year, she starred with Bogart, Edward G. Robinson, and Lionel Barrymore in L'isola di corallo (1948). The crime drama was even more of a nail biter than her previous film.In 1950, Lauren starred in Le foglie d'oro (1950), a drama set in 1894. It was a film of note because she appeared without her husband - her co-star was Gary Cooper. In 1953, Lauren appeared in her first comedy as Schatze Page in Come sposare un milionario (1953). The film, with co-stars Marilyn Monroe and Betty Grable, was a smash hit all across the 

In [0]:
# Kolumna z czasem w formacie epoch
from pyspark.sql.functions import *
namesWithTimeDf = namesDf.withColumn("execution_time_epoch", unix_timestamp(current_timestamp()))

# Kolumna przeliczenia wzrostu na stopy
namesWithFeet = namesWithTimeDf.withColumn("Height_feet", round(col("height") / 30.48, 2))

#Najpopularniejsze imie
nameCounts = namesDf.groupBy("name").count() # grupowanie po kolumnie i liczenie ile razy występuje 
mostPopularName = nameCounts.orderBy(desc("count")).limit(1) #sortowanie malejąco i wybiera najpopularniejsze imie
display(mostPopularName) #David Campbell występuje 4 razy

#Obliczanie wieku
#Parsowanie dat jeśli są (yyyy-MM-dd lub dd.MM.yyyy)
parsed_birth = when(col("date_of_birth").contains("."), to_date(col("date_of_birth"), "dd.MM.yyyy")) \
               .otherwise(to_date(col("date_of_birth"), "yyyy-MM-dd"))
parsed_death = when(col("date_of_death").contains("."), to_date(col("date_of_death"), "dd.MM.yyyy")) \
               .otherwise(to_date(col("date_of_death"), "yyyy-MM-dd"))

# np. "1961 in Spain"
birth_year = regexp_extract(col("date_of_birth"), r"(\d{4})", 1).cast("int")
death_year = regexp_extract(col("date_of_death"), r"(\d{4})", 1).cast("int")
current_year = year(current_date())

# Obliczanie wieku
namesWithAge = namesWithFeet.withColumn(
    "age",
    when(parsed_birth.isNotNull(),
         floor(
             datediff(
                 coalesce(parsed_death, current_date()),
                 parsed_birth
             ) / 365.25
         )
    ).otherwise(
        when(birth_year.isNotNull(),
             coalesce(death_year, current_year) - birth_year
        )
    )
)

#usuwanie kolumn
namesCleaned = namesWithAge.drop("bio", "death_details")

#przekształcanie naazwy: usuwamy _ i każde słowo zaczyna się wielką literą
namesRenamed = namesCleaned

for col_name in namesCleaned.columns:
    new_col_name = ' '.join([word.capitalize() for word in col_name.split('_')])
    namesRenamed = namesRenamed.withColumnRenamed(col_name, new_col_name)

#sortowanie po imieniu rosnąco
namesSorted = namesRenamed.orderBy("Name")

#Podczas wykonywania zadań sprawdź plan wykonania `explain()`
namesSorted.explain()

# Wyświetl tylko 10 wierszy
display(namesSorted.limit(10))

name,count
David Campbell,4


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Name#5821 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Name#5821 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=2304]
      +- Project [imdb_name_id#441 AS Imdb Name Id#5802, name#442 AS Name#5821, birth_name#443 AS Birth Name#5840, height#444 AS Height#5859, birth_details#446 AS Birth Details#5878, date_of_birth#447 AS Date Of Birth#5897, place_of_birth#448 AS Place Of Birth#5916, date_of_death#450 AS Date Of Death#5935, place_of_death#451 AS Place Of Death#5954, reason_of_death#452 AS Reason Of Death#5973, spouses_string#453 AS Spouses String#5992, spouses#454 AS Spouses#6011, divorces#455 AS Divorces#6030, spouses_with_children#456 AS Spouses With Children#6049, children#457 AS Children#6068, 1742644678 AS Execution Time Epoch#6087L, round((cast(height#444 as double) / 30.48), 2) AS Height Feet#6106, CASE WHEN isnotnull(CASE WHEN Contains(date_of_birth#447, .) THEN cast(gettimestamp(date_of_birth#44

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

display(namesDf.limit(10))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,$ 2250,null,null,null,7.0,7.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,$ 45000,null,null,null,25.0,3.0
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0
tt0002199,"From the Manger to the Cross or, Jesus of Nazareth","From the Manger to the Cross or, Jesus of Nazareth",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,Gene Gauntier,Kalem Company,"R. Henderson Bland, Percy Dyer, Gene Gauntier, Alice Hollister, Samuel Morgan, James D. Ainsley, Robert G. Vignola, George Kellog, J.P. McGowan","An account of the life of Jesus Christ, based on the books of the New Testament: After Jesus' birth is foretold to his parents, he is born in Bethlehem, and is visited by shepherds and wise...",05.lip,484,null,null,null,null,13.0,5.0
tt0002423,Madame DuBarry,Madame DuBarry,1919,26.11.1919,"Biography, Drama, Romance",85,Germany,German,Ernst Lubitsch,"Norbert Falk, Hanns Kräly",Projektions-AG Union (PAGU),"Pola Negri, Emil Jannings, Harry Liedtke, Eduard von Winterstein, Reinhold Schünzel, Else Berna, Fred Immler, Gustav Czimeg, Karl Platen, Bernhard Goetzke, Magnus Stifter, Paul Biensfeldt, Willy Kaiser-Heyl, Alexander Ekert, Robert Sortsch-Pla","The story of Madame DuBarry, the mistress of Louis XV of France, and her loves in the time of the French revolution.",06.sie,753,null,null,null,null,12.0,9.0
tt0002445,Quo Vadis?,Quo Vadis?,1913,01.03.1913,"Drama, History",120,Italy,Italian,Enrico Guazzoni,"Henryk Sienkiewicz, Enrico Guazzoni",Società Italiana Cines,"Amleto Novelli, Gustavo Serena, Carlo Cattaneo, Amelia Cattaneo, Lea Giunchi, Bruto Castellani, Augusto Mastripietri, Cesare Moltini, Olga Brandini, Ignazio Lupi, Giovanni Gizzi, Lia Orlandini, Matilde Guillaume, Ida Carloni Talli, Giuseppe Gambardella",An epic Italia

In [0]:
# Kolumna z czasem w formacie epoch
from pyspark.sql.functions import *
namesWithTimeDf = namesDf.withColumn("execution_time_epoch", unix_timestamp(current_timestamp()))

#obliczanie lat od publikacji
namesWithYearsSince = namesWithTimeDf.withColumn(
    "YearsSinceRelease",
    year(current_date()) - col("year").cast("int")
)

#dodawanie kolumny z budżetem (bez waluty)
namesWithNumericBudget = namesWithYearsSince.withColumn(
    "budget_clean",
    regexp_replace(col("budget"), "[^0-9]", "").cast("int")
)

#usuwanie wierszy z null
namesNoNulls = namesWithNumericBudget.na.drop()
namesNoNulls.explain()

display(namesNoNulls.limit(10))

== Physical Plan ==
*(1) Project [imdb_title_id#6514, title#6515, original_title#6516, year#6517, date_published#6518, genre#6519, duration#6520, country#6521, language#6522, director#6523, writer#6524, production_company#6525, actors#6526, description#6527, avg_vote#6528, votes#6529, budget#6530, usa_gross_income#6531, worlwide_gross_income#6532, metascore#6533, reviews_from_users#6534, reviews_from_critics#6535, 1742645347 AS execution_time_epoch#7240L, (2025 - cast(year#6517 as int)) AS YearsSinceRelease#7264, cast(regexp_replace(budget#6530, [^0-9], , 1) as int) AS budget_clean#7289]
+- *(1) Filter atleastnnonnulls(24, imdb_title_id#6514, title#6515, original_title#6516, year#6517, date_published#6518, genre#6519, duration#6520, country#6521, language#6522, director#6523, writer#6524, production_company#6525, actors#6526, description#6527, avg_vote#6528, votes#6529, budget#6530, usa_gross_income#6531, worlwide_gross_income#6532, metascore#6533, reviews_from_users#6534, reviews_from

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,execution_time_epoch,YearsSinceRelease,budget_clean
tt0017136,Metropolis,Metropolis,1927,01.10.1928,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rogge, Fritz Rasp, Theodor Loos, Erwin Biswanger, Heinrich George, Brigitte Helm","In a futuristic city sharply divided between the working class and the city planners, the son of the city's mastermind falls in love with a working class prophet who predicts the coming of a savior to mediate their differences.",08.mar,156076,DEM 6000000,$ 1236166,$ 1349711,98.0,495.0,208.0,1742645347,98,6000000
tt0021749,Luci della città,City Lights,1931,02.04.1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, Al Ernest Garcia, Hank Mann, Charles Chaplin","With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.",08.maj,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0,1742645347,94,1500000
tt0027977,Tempi moderni,Modern Times,1936,12.03.1937,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergman, Tiny Sandford, Chester Conklin, Hank Mann, Stanley Blystone, Al Ernest Garcia, Richard Alexander, Cecil Reynolds, Mira McKinney, Murdock MacQuarrie, Wilfred Lucas, Edward LeSaint, Fred Malatesta",The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,08.maj,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0,1742645347,89,1500000
tt0029453,Il bandito della Casbah,Pépé le Moko,1937,22.10.1937,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fernand Charpin, Lucas Gridoux, Gilbert Gil, Marcel Dalio, Charles Granval, Gaston Modot, René Bergeron, Paul Escoffier, Roger Legris, Jean Témerson, Robert Ozanne, Philippe Richard","A wanted gangster is both king and prisoner of the Casbah. He is protected from arrest by his friends, but is torn by his desire for freedom outside. A visiting Parisian beauty may just tempt his fate.",07.lip,6180,$ 60000,$ 155895,$ 155895,98.0,46.0,55.0,1742645347,88,60000
tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,30.11.1938,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti, Eddie Collins, Pinto Colvig, Marion Darlington, Billy Gilbert, Otis Harlan, Lucille La Verne, James MacDonald, Scotty Mattraw, Moroni Olsen, Purv Pullen, Harry Stockwell, Bill Thompson","Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",07.cze,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,1742645347,88,1499000
tt0031381,Via col vento,Gone with the Wind,1939,12.03.1949,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh, Evelyn Keyes, Ann Rutherford, George Reeves, Fred Crane, Hattie McDaniel, Oscar Polk, Butterfly McQueen, Victor Jory, Everett Brown, Howard Hickman, Alicia Rhett, Leslie Howard",A manipulative woman and a roguish man conduct a turbulent romance during the American Civil War and Reconstruction periods.,08.sty,283975,$ 3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,1742645347,86,3977000
tt0031679,Mr. Smith va a Washington,Mr

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

display(namesDf.limit(10))

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,null,null,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,null,null,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,null,null,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,null,null,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,null,null,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,null,null,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,null,null,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0
tt0002199,5.7,484,5.8,6.0,33,15,48,80,123,77,36,20,18,34,null,null,5.6,35.0,5.6,177.0,5.8,168.0,5.7,354.0,null,null,5.6,32.0,5.7,159.0,5.8,151.0,5.8,31.0,null,null,6.3,3.0,5.1,13.0,6.5,15.0,5.5,67.0,5.9,173.0,5.6,181.0
tt0002423,6.8,753,6.8,7.0,80,65,105,209,142,80,27,13,4,28,3.0,1.0,6.4,59.0,6.7,287.0,7.0,276.0,6.6,530.0,3.0,1.0,6.4,41.0,6.6,240.0,6.8,238.0,7.4,93.0,null,null,6.5,16.0,7.2,40.0,8.2,34.0,6.2,88.0,6.6,139.0,6.8,455.0
tt0002445,6.2,273,6.2,6.0,15,8,30,74,66,40,22,6,2,10,6.0,1.0,5.9,34.0,6.2,99.0,6.3,69.0,6.2,191.0,6.0,1.0,5.8,29.0,6.2,85.0,6.3,64.0,6.3,21.0,null,null,6.5,5.0,6.3,11.0,6.2,5.0,5.7,41.0,5.8,40.0,6.2,154.0
tt0002452,6.7,198,7.1,7.0,53,7,23,35,30,24,12,5,3,6,null,null,7.4,28.0,6.7,104.0,6.5,39.0,6.7,153.0,null,null,7.3,24.0,6.6,92.0,6.5,33.0,7.0,19.0,null,null,8.3,3.0,6.8,10.0,6.8,6.0,5.4,25.0,6.7,25.0,6.8,135.0
tt0002461,5.5,225,5.4,6.0,9,6,22,34,53,40,23,8,4,26,null,null,5.9,11.0,5.3,95.0,5.8,72.0,5.5,168.0,null,null,5.9,11.0,5.2,88.0,5.8,63.0,7.7,15.0,null,null,null,null,7.2,6.0,9.7,9.0,5.4,37.0,5.8,109.0,5.4,68.0


In [0]:
# Kolumna z czasem w formacie epoch
from pyspark.sql.functions import *
namesWithTimeDf = namesDf.withColumn("execution_time_epoch", unix_timestamp(current_timestamp()))

#usuwanie wierszy z null
cleanedDf = namesWithTimeDf.na.drop()

#porównanie dziewczyn i chłopców
avg_male_score = cleanedDf.select(avg("males_allages_votes")).first()[0]
avg_female_score = cleanedDf.select(avg("females_allages_votes")).first()[0]

print(f"Średnia ocena chłopców: {avg_male_score:.2f}")
print(f"Średnia ocena dziewczyn: {avg_female_score:.2f}")

#zmiana na typ long
df = cleanedDf.withColumn("total_votes", col("total_votes").cast("long"))

df.explain()
display(df.limit(10))

Średnia ocena chłopców: 26653.25
Średnia ocena dziewczyn: 6107.63
== Physical Plan ==
*(1) Project [imdb_title_id#7545, weighted_average_vote#7546, cast(total_votes#7547 as bigint) AS total_votes#11901L, mean_vote#7548, median_vote#7549, votes_10#7550, votes_9#7551, votes_8#7552, votes_7#7553, votes_6#7554, votes_5#7555, votes_4#7556, votes_3#7557, votes_2#7558, votes_1#7559, allgenders_0age_avg_vote#7560, allgenders_0age_votes#7561, allgenders_18age_avg_vote#7562, allgenders_18age_votes#7563, allgenders_30age_avg_vote#7564, allgenders_30age_votes#7565, allgenders_45age_avg_vote#7566, allgenders_45age_votes#7567, males_allages_avg_vote#7568, ... 26 more fields]
+- *(1) Filter atleastnnonnulls(49, imdb_title_id#7545, weighted_average_vote#7546, total_votes#7547, mean_vote#7548, median_vote#7549, votes_10#7550, votes_9#7551, votes_8#7552, votes_7#7553, votes_6#7554, votes_5#7555, votes_4#7556, votes_3#7557, votes_2#7558, votes_1#7559, allgenders_0age_avg_vote#7560, allgenders_0age_votes#

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,execution_time_epoch
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0,1742646321
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0,1742646321
tt0003740,7.1,3073,6.5,7.0,285,301,591,727,443,199,85,27,18,397,6.0,3.0,7.0,393.0,7.0,1126.0,7.2,1006.0,7.1,2149.0,6.0,2.0,7.0,323.0,7.0,976.0,7.3,799.0,6.9,402.0,6.0,1.0,6.8,67.0,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0,1742646321
tt0004972,6.3,22213,6.4,7.0,3661,1741,3314,3963,2876,1928,978,701,577,2474,4.7,14.0,6.0,3183.0,6.3,8861.0,6.7,4901.0,6.4,14818.0,4.7,12.0,6.1,2623.0,6.4,7669.0,6.8,4104.0,5.7,2417.0,6.0,1.0,5.4,517.0,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0,1742646321
tt0005680,5.9,130,5.9,6.0,4,3,8,20,52,27,8,5,2,1,5.0,3.0,5.8,19.0,5.9,44.0,5.9,50.0,5.9,102.0,5.0,2.0,5.8,16.0,5.8,40.0,6.0,43.0,5.9,12.0,5.0,1.0,5.5,2.0,6.5,4.0,6.2,5.0,5.7,50.0,5.9,26.0,5.8,87.0,1742646321
tt0006206,7.3,4166,6.7,7.0,620,439,763,872,484,232,140,40,31,545,6.5,7.0,7.1,517.0,7.3,1661.0,7.3,1294.0,7.3,2835.0,6.8,6.0,7.1,414.0,7.3,1384.0,7.3,976.0,7.2,670.0,4.0,1.0,6.9,98.0,7.1,256.0,7.7,306.0,6.8,204.0,7.5,1350.0,7.2,1945.0,1742646321
tt0006864,7.8,13875,7.8,8.0,3477,2230,3214,2249,1179,605,340,181,133,267,7.9,8.0,7.8,1795.0,7.7,5451.0,7.8,3667.0,7.8,9441.0,8.0,7.0,7.9,1498.0,7.7,4734.0,7.8,2970.0,7.5,1632.0,7.0,1.0,7.2,276.0,7.4,660.0,8.0,662.0,7.5,321.0,7.7,4286.0,7.8,5954.0,1742646321
tt0009611,7.3,5895,6.7,7.0,552,564,1303,1742,724,228,69,35,22,656,5.0,4.0,7.3,833.0,7.3,2162.0,7.3,1730.0,7.3,4074.0,4.7,3.0,7.3,729.0,7.3,1888.0,7.3,1340.0,7.4,746.0,6.0,1.0,7.4,96.0,7.3,257.0,7.6,380.0,6.8,283.0,7.3,1734.0,7.3,2694.0,1742646321
tt0010323,8.1,55601,7.9,8.0,11426,11262,15971,8883,3517,1562,725,483,348,1424,8.5,47.0,8.1,10734.0,8.1,23386.0,8.0,8271.0,8.1,35834.0,8.5,38.0,8.1,8304.0,8.0,19321.0,8.0,6821.0,8.1,7660.0,8.8,7.0,8.1,2236.0,8.2,3807.0,8.1,1341.0,7.5,559.0,8.0,13136.0,8.1,25399.0,1742646321
tt0011130,7.0,4753,7.1,7.0,393,335,965,1666,816,326,107,54,35,56,7.0,4.0,6.9,546.0,7.0,1830.0,7.0,1428.0,7.0,3253.0,6.0,3.0,6.9,415.0,6.9,1541.0,7.0,1207.0,7.2,606.0,10.0,1.0,6.8,128.0,7.3,269.0,7.2,202.0,6.9,239.0,7.1,1733.0,6.9,1815.0,1742646321


Zadanie 2
Przejdź przez Spark UI i opisz w kilku zdaniach co można znaleźć w każdym z elementów Spark UI.



*Jobs – pokazuje listę wszystkich zadań uruchomionych w Spark'u z ich statusem i czasem działania.

*Stages – przedstawia szczegóły każdego etapu zadania, w tym liczbę tasków, czas i błędy.

*Storage – zawiera informacje o danych przechowywanych w pamięci.

*Environment – pokazuje konfigurację Sparka, tj. parametry i zmienne systemowe.

*Executors – prezentuje statystyki tj. pamięć, CPU, I/O oraz ich status.

*SQL / DataFrame – pokazuje historię wykonanych zapytań SQL i operacji na DataFrame’ach z ich planami wykonania.

*JDBC/ODBC Server – monitoruje zapytania przesyłane do Sparka przez zewnętrzne narzędzia.

*Structured Streaming – zawiera informacje o aktywnych i zakończonych streamach w przypadku analizy danych strumieniowych.



Zadanie 3
Do jednej z Dataframe dołóż transformacje groupBy i porównaj, jak wygląda plan wykonania.



In [0]:
grouped  = namesNoNulls.groupBy("title").count()

#plan
grouped .explain()
display(grouped.limit(10))

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[title#6515], functions=[finalmerge_count(merge count#12201L) AS count(1)#12196L])
   +- Exchange hashpartitioning(title#6515, 200), ENSURE_REQUIREMENTS, [plan_id=3842]
      +- HashAggregate(keys=[title#6515], functions=[partial_count(1) AS count#12201L])
         +- Project [title#6515]
            +- Filter atleastnnonnulls(24, imdb_title_id#6514, title#6515, original_title#6516, year#6517, date_published#6518, genre#6519, duration#6520, country#6521, language#6522, director#6523, writer#6524, production_company#6525, actors#6526, description#6527, avg_vote#6528, votes#6529, budget#6530, usa_gross_income#6531, worlwide_gross_income#6532, metascore#6533, reviews_from_users#6534, reviews_from_critics#6535, (2025 - cast(year#6517 as int)), cast(regexp_replace(budget#6530, [^0-9], , 1) as int))
               +- FileScan csv [imdb_title_id#6514,title#6515,original_title#6516,year#6517,date_published#6518,genre

title,count
Casablanca,1
Pandora,1
Il buio oltre la siepe,1
Il gigante,1
My Fair Lady,1
Per un pugno di dollari,1
Quarto potere,1
Via col vento,1
Tutti per uno,1
Tempi moderni,1
